In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import json
from your_finance_module import create_finance_agent_graph  # Import your graph creation function

app = FastAPI()

class EmailRequest(BaseModel):
    userId: str
    emailContent: str
    prompt: str

async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    inputs = {"messages": [("human", query)]}

    async def stream_updates():
        try:
            async for chunk in graph.astream(inputs, stream_mode="updates"):
                for node, values in chunk.items():
                    yield f"data: {json.dumps({'node': node, 'values': values})}\n\n"
        except Exception as e:
            yield f"data: {json.dumps({'error': str(e)})}\n\n"
        finally:
            yield "data: [DONE]\n\n"

    return StreamingResponse(stream_updates(), media_type="text/event-stream")

@app.post("/generate_response")
async def generate_response(request: EmailRequest):
    return await finance_agent_generator(request.prompt)

@app.get("/getUserId")
async def get_user_id():
    return {"userId": "user1"}

@app.get("/getUserConfig/{user_id}")
async def get_user_config(user_id: str):
    return {
        "userId": user_id,
        "buttons": [
            {
                "label": "Financial Analysis",
                "icon": "CalculatorRegular",
                "apiEndpoint": "http://localhost:8000/generate_response",
                "description": "Generate a financial analysis based on the email content.",
            }
        ]
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    initial_state = AgentState(
        query=query,
        step="",
        final_response=""
        # Add other fields as necessary
    )

    async def stream_updates():
        current_state = initial_state
        async for chunk in graph.astream(current_state, stream_mode="updates"):
            for node, new_state in chunk.items():
                if isinstance(new_state, AgentState):
                    if new_state.step != current_state.step:
                        yield f"data: {json.dumps({'type': 'step', 'content': new_state.step})}\n\n"
                    if new_state.final_response and not current_state.final_response:
                        yield f"data: {json.dumps({'type': 'final', 'content': new_state.final_response})}\n\n"
                        yield "data: [DONE]\n\n"
                        return
                    current_state = new_state

    return StreamingResponse(stream_updates(), media_type="text/event-stream")